# <center>WORKING WITH TABLE DATA WITH R</center>
<img src="../elem/caldiss_symbol_square.png" width="200">


<i><center>Kristian Gade Kjelmann</center></i>
<i><center>March 5th 2020</center><i>

## Disposition

- Indlæs og udforsk tabeldata
- Subsetting
- Summary statistics
- R til datahåndtering - dplyr/tidyverse
    - Rekodningsbasics
    - Filter og selekter
- Numeriske variable
- Tekst variable
- Kategoriske variable

In [26]:
library(readr)

ess_data <- read_csv("https://github.com/CALDISS-AAU/workshop_r-table-data/raw/master/data/ess2014_mainsub_p1.csv")

Parsed with column specification:
cols(
  idno = col_double(),
  ppltrst = col_character(),
  polintr = col_character(),
  vote = col_character(),
  lrscale = col_character(),
  happy = col_character(),
  health = col_character(),
  cgtsday = col_double(),
  cgtsmke = col_character(),
  alcfreq = col_character(),
  brncntr = col_character(),
  height = col_double(),
  weight = col_double(),
  gndr = col_character(),
  yrbrn = col_double(),
  edlvddk = col_character(),
  marsts = col_character(),
  polpartvt = col_character()
)



In [27]:
head(ess_data)

idno,ppltrst,polintr,vote,lrscale,happy,health,cgtsday,cgtsmke,alcfreq,brncntr,height,weight,gndr,yrbrn,edlvddk,marsts,polpartvt
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>
921018,6,Hardly interested,Not eligible to vote,4,9,Very good,10,I smoke but not every day,2-3 times a month,Yes,178,64,Male,1990,Folkeskole 6.-8. klasse,None of these (NEVER married or in legally registered civil union),[NA] Not applicable
921026,8,Quite interested,Yes,4,8,Very good,NA,I have never smoked,Several times a week,Yes,172,64,Female,1948,"Mellemlang videregående uddannelse af 3-4 års varighed. Professionsbachelorer,",Widowed/civil partner died,[1] Socialdemokraterne - the Danish social democrats
921034,8,Quite interested,Yes,7,8,Good,NA,I don't smoke now but I used to,Every day,Yes,176,87,Male,1957,"Kort videregående uddannelse af op til 2-3 års varighed, F.eks. Erhvervsakadem",Not applicable,"[7] Venstre, Danmarks Liberale Parti - Venstre"
921181,9,Quite interested,Yes,5,9,Fair,NA,I don't smoke now but I used to,Once a week,Yes,194,102,Male,1956,"Faglig uddannelse (håndværk, handel, landbrug mv.), F.eks. Faglærte, Social-",Not applicable,[2] Det Radikale Venstre - Danish Social-Liberal Party
921204,9,Hardly interested,Yes,7,8,Good,NA,I don't smoke now but I used to,Once a week,No,157,48,Female,1941,"Kort videregående uddannelse af op til 2-3 års varighed, F.eks. Erhvervsakadem",Not applicable,[NA] Don't know
921262,8,Hardly interested,Yes,7,8,Very good,NA,I have never smoked,2-3 times a month,Yes,180,93,Male,1987,"Mellemlang videregående uddannelse af 3-4 års varighed. Professionsbachelorer,",None of these (NEVER married or in legally registered civil union),"[7] Venstre, Danmarks Liberale Parti - Venstre"


In [28]:
summary(ess_data)

      idno          ppltrst            polintr              vote          
 Min.   :921018   Length:751         Length:751         Length:751        
 1st Qu.:928301   Class :character   Class :character   Class :character  
 Median :935845   Mode  :character   Mode  :character   Mode  :character  
 Mean   :935664                                                           
 3rd Qu.:942857                                                           
 Max.   :950516                                                           
                                                                          
   lrscale             happy              health             cgtsday      
 Length:751         Length:751         Length:751         Min.   :  0.00  
 Class :character   Class :character   Class :character   1st Qu.:  5.00  
 Mode  :character   Mode  :character   Mode  :character   Median : 10.00  
                                                          Mean   : 11.51  
                         

In [29]:
colnames(ess_data)

[1] "idno"      "ppltrst"   "polintr"   "vote"      "lrscale"   "happy"    
 [7] "health"    "cgtsday"   "cgtsmke"   "alcfreq"   "brncntr"   "height"   
[13] "weight"    "gndr"      "yrbrn"     "edlvddk"   "marsts"    "polpartvt"

In [30]:
head(ess_data$gndr)

[1] "Male"   "Female" "Male"   "Male"   "Female" "Male"

In [31]:
ess_data[c(1:5), c("gndr", "alcfreq")]

gndr,alcfreq
<chr>,<chr>
Male,2-3 times a month
Female,Several times a week
Male,Every day
Male,Once a week
Female,Once a week


In [42]:
# Basic R rekodning
ess_snip <- head(ess_data, 100)

ess_snip[which(ess_data$height > 190), "height"] <- 99999

head(ess_snip, 10)

idno,ppltrst,polintr,vote,lrscale,happy,health,cgtsday,cgtsmke,alcfreq,brncntr,height,weight,gndr,yrbrn,edlvddk,marsts,polpartvt
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>
921018,6,Hardly interested,Not eligible to vote,4,9,Very good,10,I smoke but not every day,2-3 times a month,Yes,178,64,Male,1990,Folkeskole 6.-8. klasse,None of these (NEVER married or in legally registered civil union),[NA] Not applicable
921026,8,Quite interested,Yes,4,8,Very good,NA,I have never smoked,Several times a week,Yes,172,64,Female,1948,"Mellemlang videregående uddannelse af 3-4 års varighed. Professionsbachelorer,",Widowed/civil partner died,[1] Socialdemokraterne - the Danish social democrats
921034,8,Quite interested,Yes,7,8,Good,NA,I don't smoke now but I used to,Every day,Yes,176,87,Male,1957,"Kort videregående uddannelse af op til 2-3 års varighed, F.eks. Erhvervsakadem",Not applicable,"[7] Venstre, Danmarks Liberale Parti - Venstre"
921181,9,Quite interested,Yes,5,9,Fair,NA,I don't smoke now but I used to,Once a week,Yes,99999,102,Male,1956,"Faglig uddannelse (håndværk, handel, landbrug mv.), F.eks. Faglærte, Social-",Not applicable,[2] Det Radikale Venstre - Danish Social-Liberal Party
921204,9,Hardly interested,Yes,7,8,Good,NA,I don't smoke now but I used to,Once a week,No,157,48,Female,1941,"Kort videregående uddannelse af op til 2-3 års varighed, F.eks. Erhvervsakadem",Not applicable,[NA] Don't know
921262,8,Hardly interested,Yes,7,8,Very good,NA,I have never smoked,2-3 times a month,Yes,180,93,Male,1987,"Mellemlang videregående uddannelse af 3-4 års varighed. Professionsbachelorer,",None of these (NEVER married or in legally registered civil union),"[7] Venstre, Danmarks Liberale Parti - Venstre"
921319,8,Very interested,Not eligible to vote,8,Extremely happy,Fair,10,I smoke daily,Once a week,Yes,185,82,Male,1997,Folkeskole 9.-10. klasse,None of these (NEVER married or in legally registered civil union),[NA] Not applicable
921327,9,Quite interested,Yes,2,9,Good,NA,I don't smoke now but I used to,Once a week,Yes,182,97,Male,1940,"Faglig uddannelse (håndværk, handel, landbrug mv.), F.eks. Faglærte, Social-",Not applicable,[9] Enhedslisten - Unity List - The Red-Green Alliance
921335,8,Hardly interested,Yes,9,9,Good,NA,I have never smoked,Several times a week,Yes,178,84,Male,1945,"Faglig uddannelse (håndværk, handel, landbrug mv.), F.eks. Faglærte, Social-",Not applicable,[10] Andet - other
